## Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pickle

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Loading the Dataset

In [3]:
car=pd.read_csv('/content/drive/MyDrive/Car_Price_Prediction/cardekho.csv')
car.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage(km/ltr/kg),engine,max_power,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.40,1248.0,74,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14,1498.0,103.52,5.0
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.70,1497.0,78,5.0
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.00,1396.0,90,5.0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.10,1298.0,88.2,5.0


## Data Cleaning and Preprocessing

In [4]:
car.describe()

,year,selling_price,km_driven,mileage(km/ltr/kg),engine,seats
count,8128.000000,8.128000e+03,8.128000e+03,7907.000000,7907.000000,7907.000000
mean,2013.804011,6.382718e+05,6.981951e+04,19.418783,1458.625016,5.416719
std,4.044249,8.062534e+05,5.655055e+04,4.037145,503.916303,0.959588
min,1983.000000,2.999900e+04,1.000000e+00,0.000000,624.000000,2.000000
25%,2011.000000,2.549990e+05,3.500000e+04,16.780000,1197.000000,5.000000
50%,2015.000000,4.500000e+05,6.000000e+04,19.300000,1248.000000,5.000000
75%,2017.000000,6.750000e+05,9.800000e+04,22.320000,1582.000000,5.000000
max,2020.000000,1.000000e+07,2.360457e+06,42.000000,3604.000000,14.000000


In [5]:
car.isnull().sum()

name                    0
year                    0
selling_price           0
km_driven               0
fuel                    0
seller_type             0
transmission            0
owner                   0
mileage(km/ltr/kg)    221
engine                221
max_power             215
seats                 221
dtype: int64

In [6]:
car_clean = car.copy()
car_clean = car_clean.drop_duplicates()

In [7]:
car_clean = car_clean.dropna()

In [8]:
car_clean = car_clean.reset_index(drop=True)
print(car_clean.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6718 entries, 0 to 6717
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   name                6718 non-null   object 
 1   year                6718 non-null   int64  
 2   selling_price       6718 non-null   int64  
 3   km_driven           6718 non-null   int64  
 4   fuel                6718 non-null   object 
 5   seller_type         6718 non-null   object 
 6   transmission        6718 non-null   object 
 7   owner               6718 non-null   object 
 8   mileage(km/ltr/kg)  6718 non-null   float64
 9   engine              6718 non-null   float64
 10  max_power           6718 non-null   object 
 11  seats               6718 non-null   float64
dtypes: float64(3), int64(3), object(6)
memory usage: 629.9+ KB
None


## Encoding categorical columns to numerical values


In [9]:
car_clean['fuel'] = car_clean['fuel'].replace({
    'Diesel': 1, 'Petrol': 2, 'LPG': 3, 'CNG': 4
})
car_clean['transmission'] = car_clean['transmission'].replace({
    'Manual': 1, 'Automatic': 2
})
car_clean['seller_type'] = car_clean['seller_type'].replace({
    'Individual': 1, 'Dealer': 2, 'Trustmark Dealer': 3
})
car_clean['owner'] = car_clean['owner'].replace({
    'First Owner': 1, 'Second Owner': 2, 'Third Owner': 3,
    'Fourth & Above Owner': 4, 'Test Drive Car': 5
})

In [10]:
car_clean['max_power'] = pd.to_numeric(car_clean['max_power'], errors='coerce')
car_clean = car_clean.dropna(subset=['max_power'])

In [11]:
csv_path = 'car_price.csv'
car_clean.to_csv(csv_path, index=False)

print(f'Dataset saved to {csv_path}')

Dataset saved to car_price.csv


In [12]:
car_clean.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage(km/ltr/kg),engine,max_power,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,1,1,1,1,23.40,1248.0,74.00,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,1,1,1,2,21.14,1498.0,103.52,5.0
2,Honda City 2017-2020 EXi,2006,158000,140000,2,1,1,3,17.70,1497.0,78.00,5.0
3,Hyundai i20 Sportz Diesel,2010,225000,127000,1,1,1,1,23.00,1396.0,90.00,5.0
4,Maruti Swift VXI BSIII,2007,130000,120000,2,1,1,1,16.10,1298.0,88.20,5.0


## EDA

In [ ]:
fig = px.scatter(car_clean, x='year', y='selling_price', color='fuel',
                 title='Selling Price vs Year by Fuel Type')
fig.show()

In [ ]:
fig = px.box(car_clean, x='fuel', y='selling_price', color='fuel',
             title='Selling Price Distribution by Fuel Type')
fig.show()

In [ ]:
fig = px.histogram(car_clean, x='selling_price', nbins=50, color='fuel',
                   title='Distribution of Selling Price by Fuel Type')
fig.show()

In [ ]:
fig = px.box(car_clean, x='transmission', y='selling_price', color='transmission',
             title='Selling Price Distribution by Transmission Type')
fig.show()

In [ ]:
fig = px.scatter(car_clean, x='engine', y='selling_price', color='fuel',
                 title='Selling Price vs Engine Size by Fuel Type')
fig.show()

In [ ]:
fig = px.scatter(car_clean, x='max_power', y='selling_price', color='fuel',
                 title='Selling Price vs Max Power by Fuel Type')
fig.show()

In [ ]:
fig = px.scatter(car_clean, x='mileage(km/ltr/kg)', y='selling_price', color='fuel',
                 title='Selling Price vs Mileage by Fuel Type')
fig.show()

In [ ]:
numeric_cols = ['year', 'selling_price', 'km_driven', 'fuel', 'seller_type', 'transmission', 'owner', 'mileage(km/ltr/kg)', 'engine', 'max_power', 'seats']
corr = car_clean[numeric_cols].corr()
fig = go.Figure(data=go.Heatmap(
        z=corr.values,
        x=corr.index.values,
        y=corr.columns.values,
        colorscale='Viridis'))

fig.update_layout(title='Correlation Heatmap', xaxis_nticks=36)
fig.show()

## Train Test Split

In [ ]:
# Define features and target variable
X = car_clean[['year', 'km_driven', 'fuel', 'seller_type', 'transmission', 'owner', 'mileage(km/ltr/kg)', 'engine', 'max_power', 'seats']]
y = car_clean['selling_price']

In [ ]:
# Split the data
train_X, val_X, train_y, val_y = train_test_split(X, y, test_size=0.2, random_state=42)

## Model Building and Evaluation

### Random Forest Regressor


In [ ]:
forest_model = RandomForestRegressor(random_state=42)
forest_model.fit(train_X, train_y)

RandomForestRegressor(random_state=42)

### Prediction and Evaluation

In [ ]:
val_predictions = forest_model.predict(val_X)

In [ ]:
print("Mean Absolute Error: ", mean_absolute_error(val_y, val_predictions))
print("Mean Squared Error: ", mean_squared_error(val_y, val_predictions))
print("R2 Score: ", r2_score(val_y, val_predictions))

Mean Absolute Error:  77229.11336123511
Mean Squared Error:  17602989818.6367
R2 Score:  0.9197788467250003


### Hyperparameter Tuning using GridSearchCV


In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(estimator=forest_model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(train_X, train_y)

best_model = grid_search.best_estimator_
print("Best Model Parameters: ", grid_search.best_params_)

Fitting 3 folds for each of 324 candidates, totalling 972 fits


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning:

`max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.



Best Model Parameters:  {'max_depth': 20, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}


### Evaluating the best model

In [ ]:
best_val_predictions = best_model.predict(val_X)

In [ ]:
print("Best Model Mean Absolute Error: ", mean_absolute_error(val_y, best_val_predictions))
print("Best Model Mean Squared Error: ", mean_squared_error(val_y, best_val_predictions))
print("Best Model R2 Score: ", r2_score(val_y, best_val_predictions))

Best Model Mean Absolute Error:  77391.36689393014
Best Model Mean Squared Error:  17951306098.113106
Best Model R2 Score:  0.9181914837865481


## Saving the model

In [ ]:
with open('car_price_prediction_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)